
# makemore – Part 1: Bigram Language Model (Tutorial + Exercises)

In this notebook we will **build a tiny character‑level language model** for baby names,
inspired by Andrej Karpathy’s *makemore* project.

We’ll start with **pure counting** (a simple statistical model), then build a tiny
**neural network** that does almost the same thing but is trained with **gradient descent**.

You do **not** need the original video to follow this.  
This notebook is designed to be **self‑contained** and exercise‑driven.

You will:

- Load a dataset of names.
- Build a **bigram** model — a model of "next character given current character".
- Turn raw counts into probabilities and **sample new names**.
- Compute the **log‑likelihood** of the data under your model.
- Rebuild the model as a tiny neural network and **train it with gradient descent**.

Each section has:

1. **Concept / motivation** – what we’re doing and why.  
2. **Exercise cell** – with `### YOUR CODE HERE` and `raise NotImplementedError(...)`.  
3. **Solution cell** – a reference implementation.

Try the exercise first, then check the solution.

---
### Quick mental picture

- A **language model** is a function that sees some text and tells you how likely different next characters (or words) are.
- In this notebook, the "text" is just a baby name like `isabella` or `liam`.
- We treat every name as a sequence of characters, e.g. `isabella` → `i, s, a, b, e, l, l, a`.

So a bigram model is learning numbers like:

- “If the current character is `i`, how likely is the next character `s`?”
- “If the current character is `l`, how likely is the next character `l` again?”
- “If we just saw the special start symbol `.`, how likely is `m` as the first letter?”

Once we know **all those probabilities**, we can let the model "babble" new names by repeatedly sampling a next character.

---

### Toy example (to keep in your head)

Imagine the dataset is only three names:

```text
ana
bob
amy
```

We’d look at all consecutive character pairs (including a start `.` and end `.`):

- `.a`, `a.n`, `n.a`, `a.`
- `.b`, `b.o`, `o.b`, `b.`
- `.a`, `a.m`, `m.y`, `y.`

From just these counts we could already say things like:

- After `.`, `a` is more common than `b`.
- After `a`, `n`, `.` and `m` are all possible next characters.

Everything we do later is just a **fancier, vectorized, neural‑networkified** version of this toy story.



## Setup – imports and dataset

We’ll work with a text file `names.txt` that has one baby name per line, e.g.

```text
emma
oliver
ava
liam
...
```

Each name is a **sequence of characters**. We’ll think of this as text data and learn
a model of what character tends to come next.

First we’ll import some libraries and load the list of names into memory.

---
### What the `words` list really is

After running the code cell below, `words` is just a **plain Python list of strings**:

```python
['emma', 'oliver', 'ava', 'liam', ...]
```

You can use all normal list operations on it:

```python
words[0]      # first name
len(words)    # how many names we have
len(words[0]) # how many characters in the first name
```

Try poking at `words` a bit when you run the notebook; getting very comfortable with the raw data will make the later tensor code feel much less mysterious.


In [ ]:

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline

# Load the dataset: one name per line
words = open('names.txt', 'r').read().splitlines()
len(words), words[:10]



## Exercise 1 – Exploring the dataset

Before building any model, always **look at your data**.

We have `words`, which is just a Python list of strings.  
Let’s answer a few basic questions:

- How many names are there?
- What are some examples?
- What is the **shortest** name?
- What is the **longest** name?

This gives us a feel for the scale of the problem and typical sequence lengths.

### Task

Using the list `words`:

1. Display the first 10 names.
2. Compute:
   - the total number of names,
   - the minimum length,
   - the maximum length.

Print them in a readable way.

---
### Solution discussion

The key ideas:

- `len(words)` gives the **number of names** in the dataset.
- `words[:10]` slices the first 10 elements of the list (Python slicing).
- To talk about **lengths of names**, you take `len(w)` for each `w` in `words`:

  ```python
  lengths = [len(w) for w in words]
  ```

  Now:
  - `min(lengths)` is the length of the shortest name,
  - `max(lengths)` is the length of the longest name.

Why this matters:

- Knowing typical lengths (say 2–15 characters) helps us reason about how many prediction steps our model will usually have to make.
- Later, when we generate new names, you’ll recognize if their lengths look “reasonable” compared to the training data.


In [ ]:

# Exercise 1 – your turn

### YOUR CODE HERE
raise NotImplementedError("Exercise 1: explore the names dataset")

# Hints:
# - len(words)          -> number of names
# - len(w) for w in ... -> name lengths


In [ ]:

# Solution 1 – exploring the dataset

print("Number of names:", len(words))
print("First 10 names:", words[:10])

lengths = [len(w) for w in words]
print("Shortest name length:", min(lengths))
print("Longest   name length:", max(lengths))



## Bigram models – what are we trying to learn?

We want a model that can **generate new names** that look "name‑like".

One very simple idea is a **bigram model**:

> The probability of the next character depends only on the current character.

So the model needs to know, for example:

- After `'a'`, how likely is `'n'`, `'l'`, `' '` (end), etc.?
- After `'b'`, how likely is `'r'`, `'a'`, ...?

We’ll also introduce special **start** and **end** markers so we know where names begin and end.

- Let’s use `'<S>'` for "start" and `'<E>'` for "end" (we’ll also later use `'.'` as a start/end symbol).

For each name `w`, we will look at all consecutive character pairs:

```text
<S> a n a <E>
    ^ ^ ^ ^
    | | | |
  bigrams: (<S>, 'a'), ('a', 'n'), ('n', 'a'), ('a', '<E>')
```

Our goal in this section is to **count** how often each bigram occurs in the dataset.

---
### Why “bigram”?

A **bigram** just means “a pair of things in a row”.

Here the “things” are characters. For the name `emma` (ignoring start/end for a moment) the character bigrams are:

```text
e m
m m
m a
```

When we *do* include start/end (`.`), we get:

```text
. e
e m
m m
m a
a .
```

Our model will learn numbers like:

- \(P(\text{'m'} \mid \text{'e'})\) – “If I just saw an `e`, how likely is the next character `m`?”
- \(P(\text{'.'} \mid \text{'a'})\) – “If I just saw an `a`, how likely is the name to end now?”

We **ignore everything except the current character** when predicting the next one.  
That’s what makes it a bigram model instead of a more powerful “look at the last 3 or 10 characters” model.

Later in the course we’ll relax this and let the network remember *longer* context, but bigrams are the perfect sandbox to understand all the core ideas.



## Exercise 2 – Bigram counts with a Python dictionary

We’ll start with a simple Python dictionary `b` that maps

```python
(ch1, ch2) -> count
```

where `ch1` and `ch2` are characters (or `'<S>'` / `'<E>'`).

Algorithm:

1. Start with an empty dict `b = {}`.
2. For each word `w` in `words`:
   - Build a list of characters including start and end markers:

     ```python
     chs = ['<S>'] + list(w) + ['<E>']
     ```

   - Loop over adjacent pairs `(ch1, ch2)` using `zip(chs, chs[1:])`.
   - Increment `b[(ch1, ch2)]` by 1 (using `.get(..., 0)` for convenience).

### Task

1. Build the dictionary `b` of bigram counts.
2. Print how many **distinct** bigrams appear (`len(b)`).
3. Print the **top 20 most frequent** bigrams and their counts, sorted by decreasing count.

---
### Solution discussion

We use a Python **dictionary** `b` where:

```python
b[('a', 'n')] = 123
```

means the pair `'a'` followed by `'n'` occurred 123 times in the whole dataset.

The important pieces:

- `chs = ['<S>'] + list(w) + ['<E>']`  
  turns `"emma"` into `['<S>', 'e', 'm', 'm', 'a', '<E>']`.

- `zip(chs, chs[1:])`  
  lines up all consecutive pairs:

  ```python
  ['<S>', 'e', 'm', 'm', 'a', '<E>']
       ['e',  'm', 'm', 'a', '<E>']
  # → ('<S>', 'e'), ('e', 'm'), ('m', 'm'), ('m', 'a'), ('a', '<E>')
  ```

- `b[bigram] = b.get(bigram, 0) + 1`  
  says “look up the existing count (0 if we’ve never seen this bigram), then add 1”.

Sorting by the value (`key=lambda kv: -kv[1]`) lets us see **which character pairs are most common**.  
Those common pairs will be the ones the model leans on most when it starts generating names.


In [ ]:

# Exercise 2 – your turn: bigram counts in a Python dict

### YOUR CODE HERE
raise NotImplementedError("Exercise 2: build bigram counts dict 'b'")

# Hints:
# - Use: chs = ['<S>'] + list(w) + ['<E>']
# - Use: for ch1, ch2 in zip(chs, chs[1:]): ...
# - Use: b[bigram] = b.get(bigram, 0) + 1


In [ ]:

# Solution 2 – bigram counts in a dict

b = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

print("Number of distinct bigrams:", len(b))

# Sort by decreasing count
top20 = sorted(b.items(), key=lambda kv: -kv[1])[:20]
for (ch1, ch2), count in top20:
    print(f"{ch1!r}{ch2!r}: {count}")



## From dictionary counts to a matrix

Python dicts are nice for inspection, but we eventually want to work with **tensors**.

We’ll:

1. Build a **vocabulary** of characters.
2. Assign each character an **index**.
3. Build a **27×27 matrix** `N` of bigram counts:
   - 27 = 26 lowercase letters + 1 special symbol `'.'` that we’ll use as **both** start and end.

We will:

- Use `'.'` to represent both the beginning and the end of a word.
- Map characters to integers with a dict `stoi` ("string to index").
- Map indices back to characters with `itos` ("index to string").
- Fill `N[ix1, ix2]` with the count of how often character with index `ix2` follows `ix1`.

---
### Why bother with a 27×27 matrix?

You can think of `N` as a **big 2D table**:

- Rows = “current character”
- Columns = “next character”
- Entry `N[i, j]` = “how many times did char `j` follow char `i`?”

So if we order the characters as:

```text
. a b c d ... z
```

then `N[stoi['a'], stoi['n']]` is the number of `"an"` bigrams in the dataset.

Neural networks like working with **tensors** (multi‑dimensional arrays), not Python dicts.  
By moving to a matrix:

- We can normalise all rows in one go.
- We can visualise the whole thing as an image.
- Later we’ll literally treat this matrix as the **weights** of a neural network.

The `stoi` / `itos` pair is just a **translator** between:

- human‑friendly world: `'a'`, `'b'`, `'.'`
- tensor‑friendly world: `0, 1, 2, ...`



## Exercise 3 – Character vocabulary and bigram count matrix `N`

### Task

1. Build the set of characters present in `words` (excluding the start/end marker for now).
2. Sort them and store in `chars`.
3. Build a dict `stoi`:
   - assign index `0` to `'.'`,
   - assign indices `1..` to all characters in `chars`.
4. Build the reverse dict `itos` such that `itos[ix]` returns the character.
5. Create a tensor `N` of shape `(27, 27)` (dtype `torch.int32`) filled with zeros.
6. Loop over all words and all bigrams using `'.'` as start/end and increment the corresponding `N[ix1, ix2]`.

Afterwards, print:

- `chars`
- `stoi`
- `N.shape` and `N[0]` (counts of characters that follow `'.'`).

---
### Solution discussion

Steps you should see in your code:

1. **Build the character set**

   ```python
   chars = sorted(list(set(''.join(words))))
   ```

   - `''.join(words)` glues all names into one long string.
   - `set(...)` throws away duplicates.
   - `sorted(...)` gives you `'a'` to `'z'` in order.

2. **Index the characters**

   ```python
   stoi = {s: i + 1 for i, s in enumerate(chars)}
   stoi['.'] = 0
   itos = {i: s for s, i in stoi.items()}
   ```

   - We reserve index `0` for `'.'` (start/end),
   - Letters get indices `1..26`.

3. **Fill the matrix**

   For each bigram `(ch1, ch2)`, you do:

   ```python
   ix1 = stoi[ch1]
   ix2 = stoi[ch2]
   N[ix1, ix2] += 1
   ```

   After the loop, `N` contains exactly the same information as our dictionary from Exercise 2, just in a dense, GPU‑friendly format.


In [ ]:

# Exercise 3 – your turn: vocabulary and bigram count matrix

### YOUR CODE HERE
raise NotImplementedError("Exercise 3: build chars, stoi, itos, and N")

# Hints:
# - chars = sorted(list(set(''.join(words))))
# - stoi = {s: i+1 for i, s in enumerate(chars)}; then set stoi['.'] = 0
# - itos = {i: s for s, i in stoi.items()}
# - N = torch.zeros((27, 27), dtype=torch.int32)


In [ ]:

# Solution 3 – vocabulary and bigram count matrix

# 1. Character vocabulary
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}  # 1..26
stoi['.'] = 0                                   # 0 is the special start/end
itos = {i: s for s, i in stoi.items()}

print("chars:", chars)
print("stoi:", stoi)

# 2. Bigram count matrix
N = torch.zeros((27, 27), dtype=torch.int32)

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

print("N.shape:", N.shape)
print("Counts of characters following '.':")
print(N[0])



## Exercise 4 – Visualizing bigram counts

The matrix `N` tells us, for each character on the **rows**, how often each character on the
**columns** followed it in the dataset.

A nice way to understand these counts is to **visualize** the matrix as an image.

We’ll use `matplotlib` to plot `N` as a heatmap and overlay:

- the bigram string `itos[i] + itos[j]`,
- the count `N[i, j]`.

This lets us see which transitions are very common and which almost never happen.

### Task

1. Use `plt.imshow(N, cmap='Blues')` to show the matrix.
2. For each `i, j` in `0..26`, write:
   - the bigram string near the top of the cell in gray,
   - the count near the bottom.
3. Turn off axes for a clean image.

---
### Reading the heatmap

Each **cell** in the image corresponds to one entry `N[i, j]`:

- The **background colour** (from light to dark blue) shows how big the count is.
- The **text label** shows:
  - the bigram itself, e.g. `th` or `a.`
  - the exact count, e.g. `4321`.

Typical things you’ll notice:

- Some columns or rows are almost all dark: very common letters.
- Some pairs basically never occur in English‑like names (e.g. `qz`).
- The row for `'.'` shows which letters commonly **start** names.
- The column for `'.'` shows which letters commonly **end** names.

This kind of visual intuition is super helpful later when we compare to the neural model:  
the neural network is trying to learn a matrix whose heatmap “rhymes” with this one.


In [ ]:

# Exercise 4 – your turn: visualize N

### YOUR CODE HERE
raise NotImplementedError("Exercise 4: visualize bigram count matrix N")

# Hints:
# - Use nested for loops: for i in range(27): for j in range(27):
# - chstr = itos[i] + itos[j]
# - Use plt.text(j, i, ...)


In [ ]:

# Solution 4 – visualize bigram count matrix

plt.figure(figsize=(16, 16))
plt.imshow(N, cmap='Blues')

for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')

plt.axis('off')
plt.show()



## From counts to probabilities

Right now `N[i, j]` is a **count**: how many times did character `j` follow character `i`?

We want to interpret each row as a **probability distribution** over the next character. For row `i`:

\[
P(i \to j) = \frac{N_{ij}}{\sum_k N_{ik}}
\]

So:

- Take row `i` as a vector of counts.
- Convert to float.
- Divide by the sum of the row.

This gives a vector `p` that sums to 1: a valid discrete probability distribution.

We’ll first play with a single row, then turn the whole matrix into a probability matrix `P`.

---
### Tiny numeric example

Suppose one row of `N` (say, for the character `'a'`) looks like:

```text
[ 0, 10, 5 ]   # pretend there are only 3 possible next characters
```

- Total count = `10 + 5 = 15`.
- Probabilities:

  ```text
  [ 0/15, 10/15, 5/15 ] = [0.0, 0.666..., 0.333...]
  ```

Interpretation:

- After `'a'`, the first character never appears.
- The second character appears about **two‑thirds** of the time.
- The third character appears about **one‑third** of the time.

We’ll do this normalisation for **every row** of `N`, turning raw frequencies into proper probability distributions that always sum to 1.



## Exercise 5a – Single‑row probabilities and sampling

Let’s look at the row for the start/end symbol `'.'` (index 0).  
This row tells us: "How likely is each character as the **first** character of a name?"

### Task

1. Take `N[0]`, convert to float (`.float()`), and normalise it to sum to 1.
2. Verify that `p.sum()` is 1 (or very close, floating‑point wise).
3. Using a fixed random generator `g`, sample indices from this distribution with
   `torch.multinomial` and map them back to characters using `itos`.

---
### Solution discussion

For the start/end symbol `'.'` (index `0`):

1. `p = N[0].float()`  
   copies the first row and converts it from integers (counts) to floats.

2. `p = p / p.sum()`  
   divides each entry by the **row sum**, so that `p` now sums to 1.

   This is just the formula:

   \[
   p_j = \frac{N_{0j}}{\sum_k N_{0k}}
   \]

3. `torch.multinomial(p, num_samples=..., replacement=True)`  

   - Think of `multinomial` as “roll a weighted 27‑sided dice”.
   - It returns indices where index `j` is drawn with probability `p[j]`.

   Using `itos[ix]` turns those indices back into actual characters.

If you print a bunch of samples you should see that *more common starting letters in the dataset* (like `a`, `m`, `k`, etc.) appear more often in your drawn characters.


In [ ]:

# Exercise 5a – your turn: one row -> probabilities and sampling

### YOUR CODE HERE
raise NotImplementedError("Exercise 5a: compute p from N[0] and sample characters")

# Hints:
# - p = N[0].float()
# - p = p / p.sum()
# - g = torch.Generator().manual_seed(2147483647)
# - torch.multinomial(p, num_samples=10, replacement=True, generator=g)


In [ ]:

# Solution 5a – one row -> probabilities and sampling

# row for '.'
p = N[0].float()
p = p / p.sum()
print("p:", p)
print("sum of p:", p.sum().item())

# sample some starting characters
g = torch.Generator().manual_seed(2147483647)
sampled_ix = torch.multinomial(p, num_samples=20, replacement=True, generator=g)
print("Sampled indices:", sampled_ix.tolist())
print("Sampled chars:  ", ''.join(itos[ix.item()] for ix in sampled_ix))



## Exercise 5b – Full probability matrix `P` and sampling new names

We want to turn the whole `N` into a probability matrix `P` where **each row** sums to 1.

We’ll also add **smoothing** (add 1 to every count) so that we never get zero probabilities
for transitions that just didn’t happen in our training data.

Steps:

1. Create `P = (N + 1).float()`   (add‑one smoothing).
2. Divide each row by its row sum: `P /= P.sum(1, keepdims=True)`.

Now `P[ix]` is the probability distribution of the next character, given the current character index `ix`.

### Sampling names

We can now **sample names** from this bigram model:

1. Start from `ix = 0` (for `'.'`).
2. Sample the next index from `P[ix]` with `torch.multinomial`.
3. Convert the index to a character with `itos[ix]` and append it to an output list.
4. Repeat from step 2, using the new `ix`, until you sample `ix == 0` again (which means end of name).
5. Join the characters to form a sampled name.

### Task

1. Build the smoothed probability matrix `P` from `N`.
2. Use `P` to sample and print 10 random names.

---
### Why add 1 everywhere? (smoothing)

If a bigram **never** appears in the training data, its raw count is 0, so its probability would also be 0.

That causes two problems:

- The model will say some perfectly reasonable names have *exactly* zero probability just because we never saw them.
- In log space, `log(0)` is `-∞`, which explodes our loss.

Adding 1 to **every entry**:

```python
P = (N + 1).float()
P /= P.sum(1, keepdims=True)
```

is called **add‑one smoothing** (also “Laplace smoothing”):

- It’s as if we pretended we saw every possible bigram **once**.
- Rare or unseen bigrams get a small, but nonzero, probability.

---

### Sampling names again

The sampling loop is the same idea as Exercise 5a, but repeated:

```text
. → (sample) → first letter
(first letter) → (sample) → second letter
(second letter) → (sample) → third letter
...
until we sample '.' again
```

Each step only looks at **the current character** and uses the corresponding row of `P` as the distribution over the next one.

If your code is right, the sampled names should “feel” more name‑like than pure noise, but still obviously worse than real names — that’s the limitation of bigrams.


In [ ]:

# Exercise 5b – your turn: build P and sample names

### YOUR CODE HERE
raise NotImplementedError("Exercise 5b: build P and sample names")

# Hints:
# - P = (N + 1).float()
# - P /= P.sum(1, keepdims=True)
# - see loop description in the text above


In [ ]:

# Solution 5b – full probability matrix and sampling

# Build smoothed probability matrix
P = (N + 1).float()
P /= P.sum(1, keepdims=True)

g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    ix = 0  # start with '.'
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))



## Log‑likelihood of the data under the bigram model

So far we’ve **built** a bigram model from data by counting, and we can **sample** from it.

We’d also like a **single number** that measures *how well* the model explains the dataset.

A natural choice is the **log‑likelihood**:

- For each name, you can compute the probability that the model assigns to that exact name:
  \
  P(w) = \prod_{\text{bigrams }(c_1, c_2)\text{ in }w} P(c_2 \mid c_1)
  \
- Products of many small numbers quickly underflow, so we work with **log** probabilities:
  \
  \log P(w) = \sum \log P(c_2 \mid c_1)
  \
- For the entire dataset, we sum over all names:
  \
  \text{log\_likelihood} = \sum_{w \in \text{data}} \log P(w)
  \

Higher log‑likelihood = better model.  
People often use **average negative log‑likelihood** as a **loss** to minimise:

\[
\text{loss} = - \frac{1}{N} \sum \log P(c_2 \mid c_1)
\]

where the sum is over all bigrams in the dataset.

---
### Why products become sums (and why we care)

If a name has bigram probabilities:

```text
P = [0.4, 0.2, 0.5, 0.1]   # one for each bigram in the name
```

then

- Likelihood of that name:
  \[
  P(w) = 0.4 \times 0.2 \times 0.5 \times 0.1
  \]
- Log‑likelihood:
  \[
  \log P(w) = \log 0.4 + \log 0.2 + \log 0.5 + \log 0.1
  \]

Taking logs turns a product of many tiny numbers into a **sum of reasonably sized negatives**, which:

- is numerically safer (less underflow),
- plays nicely with calculus (sums are easier to differentiate than long products),
- matches what most deep learning libraries expect (losses are usually expressed as averages of per‑example terms).

The average negative log‑likelihood we compute in the next exercise is exactly the same quantity that appears as the **cross‑entropy loss** in classification problems.



## Exercise 6 – Log‑likelihood and average negative log‑likelihood

Using the probability matrix `P` you built in Exercise 5b, compute:

1. Total log‑likelihood `log_likelihood` of the entire dataset.
2. Total number of bigrams `n` used in this computation.
3. Average negative log‑likelihood: `nll = -log_likelihood / n`.

Use `torch.log(prob)` for log probabilities.

### Task

Fill in the loop and print the values.

---
### Solution discussion

For each bigram `(ch1, ch2)` in every word:

1. Convert characters to indices: `ix1 = stoi[ch1]`, `ix2 = stoi[ch2]`.
2. Look up the model’s probability for that transition: `prob = P[ix1, ix2]`.
3. Accumulate the log‑probability:

   ```python
   logprob = torch.log(prob)
   log_likelihood += logprob
   n += 1
   ```

At the end:

- `log_likelihood` is the sum of all \(\log P(c_2 \mid c_1)\) terms.
- `n` is the total **number of bigrams** we saw.
- The average negative log‑likelihood

  ```python
  nll = -log_likelihood / n
  ```

  is our scalar “how bad is the model?” number.

If you compare this `nll` to what you later get from the neural network model, they should be very close — that’s how we know the neural network succeeded in re‑learning the same bigram statistics.


In [ ]:

# Exercise 6 – your turn: log-likelihood

### YOUR CODE HERE
raise NotImplementedError("Exercise 6: compute log_likelihood and average NLL")

# Hints:
# - Outer loop over all words
# - Inner loop over bigrams of ['.'] + list(w) + ['.']
# - Use P[ix1, ix2] to get probability
# - Accumulate log_likelihood and n (count of bigrams)


In [ ]:

# Solution 6 – log-likelihood and average NLL

log_likelihood = 0.0
n = 0

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1

print(f"log_likelihood = {log_likelihood.item()}")
nll = -log_likelihood
print(f"total negative log-likelihood = {nll.item()}")
print(f"average NLL per bigram        = {nll.item() / n}")



## From counts to a tiny neural network

The model we built using counts and normalisation can be viewed as a **neural network** with:

- Input: the index of the current character (an integer 0..26).
- Representation: a **one‑hot vector** of length 27 (all zeros except a 1 at the current character).
- Parameters: a matrix `W` of shape `(27, 27)`.
- Output: probabilities for the next character.

Forward pass idea:

1. Convert input indices (shape `[num_examples]`) to one‑hot encodings (shape `[num_examples, 27]`).
2. Multiply by `W` to get **logits**: `logits = xenc @ W`.
3. Turn logits into (unnormalised) positive numbers by exponentiating: `counts = logits.exp()`.
4. Normalise row‑wise to get probabilities: `probs = counts / counts.sum(1, keepdims=True)`.

This `probs` has the same shape as `P`, but now it comes from **parameters** in `W` instead of counts.
We can now **learn** `W` by gradient descent instead of computing it directly from counts.

---
### Connecting the two views

You can literally think of the count‑based bigram table as a **neural network layer**:

- One‑hot input of length 27:
  - row vector: `[0, 0, 0, 1, 0, ..., 0]` means “current character is index 3”.
- Weight matrix `W` of shape `(27, 27)`:
  - row `i` contains 27 numbers controlling the scores for all possible next characters given current character `i`.

Then:

```python
xenc @ W      # pick out the row corresponding to the 1 in xenc
```

is just a fancy way of saying “look up the row for the current character”.

The only difference from the pure‑count model is:

- before, the entries of the matrix were **fixed counts**;
- now, the entries of `W` are **trainable parameters** we will adjust to make the log‑likelihood as high as possible.

This is a very gentle first example of how probabilistic models and neural networks are just two ways to talk about the **same underlying math**.



## Exercise 7 – Building a small (x, y) training set

To train a neural network, we want input–label pairs `(x, y)` where:

- `x` is the index of the current character,
- `y` is the index of the next character.

For a word `w`, we again look at `chs = ['.'] + list(w) + ['.']` and record all bigrams.

To understand shapes, let’s start with just **one word**.

### Task

1. For the *first* word in `words` only, build two Python lists `xs` and `ys`.
2. For each bigram `(ch1, ch2)`:
   - append `stoi[ch1]` to `xs`,
   - append `stoi[ch2]` to `ys`.
3. Convert `xs` and `ys` to integer tensors with `torch.tensor(...)`.
4. Print `xs`, `ys` and their shapes.

---
### Solution discussion

For a concrete example, suppose the first word is `"emma"` and we add `'.'` on both sides:

```text
['.', 'e', 'm', 'm', 'a', '.']
```

The bigrams are:

```text
('.', 'e'),
('e', 'm'),
('m', 'm'),
('m', 'a'),
('a', '.')
```

We now replace characters with their indices using `stoi`:

```python
xs = [stoi['.'], stoi['e'], stoi['m'], stoi['m'], stoi['a']]
ys = [stoi['e'], stoi['m'], stoi['m'], stoi['a'], stoi['.']]
```

After converting to tensors:

```python
xs = tensor([...])
ys = tensor([...])
```

each position `i` in `xs` / `ys` is one supervised training example:

- input `xs[i]` → “current character index”
- label `ys[i]` → “true next character index”

The rest of the notebook just scales this idea up from **one word** to **all words**.


In [ ]:

# Exercise 7 – your turn: small training set for one word

### YOUR CODE HERE
raise NotImplementedError("Exercise 7: build xs and ys for first word")

# Hints:
# - w = words[0]
# - chs = ['.'] + list(w) + ['.']
# - use zip(chs, chs[1:]) as before


In [ ]:

# Solution 7 – small training set for one word

xs, ys = [], []

w = words[0]
print("First word:", w)
chs = ['.'] + list(w) + ['.']
for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    print(ch1, ch2)
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

print("xs:", xs)
print("ys:", ys)
print("xs.shape:", xs.shape)
print("ys.shape:", ys.shape)



## Exercise 8 – One‑hot encoding and visualisation

Neural networks usually take **vectors** as input, not integer indices.  
We’ll convert indices `xs` into a 2D tensor of one‑hot vectors using:

```python
xenc = F.one_hot(xs, num_classes=27).float()
```

- Shape will be `[num_examples, 27]`.
- Each row corresponds to one bigram’s first character.

### Task

1. Use `torch.nn.functional.one_hot` to compute `xenc` from `xs` with `num_classes=27`.
2. Check its shape and dtype.
3. Visualise it with `plt.imshow(xenc)` (you should see a pattern of 1s along columns).

---
### Solution discussion

`F.one_hot(xs, num_classes=27)` turns something like:

```python
xs = tensor([0, 5, 13, 13, 1])
```

into a 2D tensor:

```text
[[1, 0, 0, ..., 0],   # index 0
 [0, 0, 0, ..., 0],   # index 5 has a 1 somewhere in the middle
 ...
]
```

Key points:

- Shape is `(num_examples, 27)`.
- Every row has exactly **one** 1 and the rest are 0s → hence “one‑hot”.
- Casting to `.float()` gives the `float32` type that neural nets expect.

When you plot it with `plt.imshow(xenc)`, each row shows a single bright pixel at the column corresponding to the input character. Later, multiplying this by `W` effectively **selects a row** of `W` for each example.


In [ ]:

# Exercise 8 – your turn: one-hot encoding and visualisation

### YOUR CODE HERE
raise NotImplementedError("Exercise 8: compute xenc and visualise")

# Hints:
# - xenc = F.one_hot(xs, num_classes=27).float()
# - xenc.shape
# - plt.imshow(xenc)


In [ ]:

# Solution 8 – one-hot encoding and visualisation

xenc = F.one_hot(xs, num_classes=27).float()
print("xenc.shape:", xenc.shape)
print("xenc.dtype:", xenc.dtype)

plt.imshow(xenc)
plt.title("One-hot encoding of first word's bigrams")
plt.xlabel("character index")
plt.ylabel("bigram example index")
plt.show()



## Exercise 9 – A tiny linear bigram model on a few examples

We’ll now treat the bigram model as a tiny one‑layer neural net:

- Input: one‑hot vectors `xenc` (shape `[num_examples, 27]`).
- Parameters: weight matrix `W` (shape `[27, 27]`).
- Output: `probs` (shape `[num_examples, 27]`) — probability distribution over next character.

Forward pass:

```python
W = torch.randn((27, 27))  # random initial weights
logits = xenc @ W          # shape: (num_examples, 27)
counts = logits.exp()      # positive
probs = counts / counts.sum(1, keepdims=True)  # row-wise normalisation
```

For now we’ll work with the small `xs, ys` from the first word only.

We’ll define the **loss** as the average negative log probability of the correct character:

```python
loss = -probs[torch.arange(num_examples), ys].log().mean()
```

This is the same as the average negative log‑likelihood we computed earlier, but now for a neural net.

### Task

1. Initialise a random weight matrix `W` of shape `(27, 27)` with a fixed random seed.
2. Compute `xenc`, `logits`, `counts`, and `probs` as above.
3. Compute and print the loss.

---
### Solution discussion

We’re now building the tiniest possible neural bigram model:

1. **Random weights**

   ```python
   g = torch.Generator().manual_seed(2147483647)
   W = torch.randn((27, 27), generator=g)
   ```

   - Each row in `W` will *eventually* behave like one row of the bigram table.
   - Right now they’re random, so predictions are bad.

2. **Forward pass**

   ```python
   xenc = F.one_hot(xs, num_classes=27).float()
   logits = xenc @ W          # shape: (num_examples, 27)
   counts = logits.exp()
   probs = counts / counts.sum(1, keepdims=True)
   ```

   - `logits` are arbitrary real numbers.
   - `exp` makes them positive (“fake counts”).
   - Dividing by row sums turns them into probabilities (“softmax”).

3. **Loss**

   ```python
   loss = -probs[torch.arange(num), ys].log().mean()
   ```

   This is exactly the average negative log‑likelihood, but now the probabilities come from `W` instead of the count matrix.

If you print the loss, it should be **a bit larger** than the loss of the count‑based model (because random `W` is very bad). Training in the next exercises will drive this loss down.


In [ ]:

# Exercise 9 – your turn: tiny linear model on a few bigrams

### YOUR CODE HERE
raise NotImplementedError("Exercise 9: build W, forward pass, and loss")

# Hints:
# - g = torch.Generator().manual_seed(2147483647)
# - W = torch.randn((27, 27), generator=g)
# - num = xs.nelement()
# - loss = -probs[torch.arange(num), ys].log().mean()


In [ ]:

# Solution 9 – tiny linear model on a few bigrams

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g)

xenc = F.one_hot(xs, num_classes=27).float()
logits = xenc @ W           # shape: (num_examples, 27)
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)

num = xs.nelement()
loss = -probs[torch.arange(num), ys].log().mean()

print("probs.shape:", probs.shape)
print("loss:", loss.item())



## Exercise 10 – One training step with autograd

So far we just computed the loss for a randomly initialised `W`.  
Now let’s **train** `W` a bit using **PyTorch autograd**.

Steps for *one* gradient descent step:

1. Create `W` with `requires_grad=True` so PyTorch tracks its gradients.
2. Forward pass to compute `loss` as before.
3. Zero the gradient: `W.grad = None`.
4. Backward pass: `loss.backward()` to fill `W.grad` with the gradient of `loss` w.r.t. `W`.
5. Update: `W.data += -learning_rate * W.grad`.

### Task

Implement one training step on the small `(xs, ys)` you built earlier and print:

- the loss before the update,
- the loss after the update (recompute the forward pass after updating `W`).

---
### Solution discussion

This is your first full **gradient descent step** in PyTorch:

1. Create `W` with `requires_grad=True` so PyTorch tracks how the loss depends on each element of `W`.
2. Do the forward pass to compute `loss`.
3. Clear old gradients with `W.grad = None`.
4. Call `loss.backward()`:
   - PyTorch walks the computation graph *backwards*,
   - fills `W.grad` with \(\frac{\partial \text{loss}}{\partial W_{ij}}\) for every entry \(W_{ij}\).

5. Update:

   ```python
   W.data += -lr * W.grad
   ```

   - If a gradient entry is positive, we subtract a bit → decreasing that weight lowers the loss.
   - If it’s negative, subtracting makes the weight bigger.

When you recompute `loss` after the update, it should be **slightly smaller**, showing that we moved in a good direction in parameter space.


In [ ]:

# Exercise 10 – your turn: one autograd training step

### YOUR CODE HERE
raise NotImplementedError("Exercise 10: one training step on small dataset")

# Hints:
# - Use a fresh W with requires_grad=True
# - Use a learning rate like 0.1 or 1.0 (just to see the effect)


In [ ]:

# Solution 10 – one autograd training step

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

def forward(xs, ys):
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    num = xs.nelement()
    loss = -probs[torch.arange(num), ys].log().mean()
    return loss

# Loss before update
loss_before = forward(xs, ys)
print("loss before:", loss_before.item())

# Backprop and update
W.grad = None
loss_before.backward()
lr = 1.0
W.data += -lr * W.grad

# Loss after update
loss_after = forward(xs, ys)
print("loss after :", loss_after.item())



## Exercise 11 – Training on the full dataset

Now we scale up from the first word to **all** words.

Steps:

1. Build bigram dataset `(xs, ys)` for **all** names:
   - Loop over all words,
   - For each, loop over bigrams of `['.'] + list(w) + ['.']`,
   - Append `stoi[ch1]` to `xs`, `stoi[ch2]` to `ys`.
2. Convert `xs` and `ys` to tensors.
3. Create weight matrix `W` with `requires_grad=True`.
4. Run a training loop for some number of steps:
   - Forward pass on all examples.
   - Compute loss as average negative log‑likelihood plus a small regularisation term, e.g. `0.01*(W**2).mean()` to discourage very large weights.
   - Zero gradients, backprop (`loss.backward()`), and update `W` with a learning rate.

We will not worry about efficiency or batching here – just a clear, simple implementation.

### Task

1. Build the full `(xs, ys)` dataset.
2. Implement a training loop for, say, 50 steps and print the loss every few steps.

---
### Solution discussion

Conceptually we just scaled up what you already did for a single word:

1. **Build the big dataset**

   - Loop over *all* words.
   - For each name, add all its bigrams to `xs` / `ys`.
   - You should end up with tens of thousands of examples.

2. **Single big forward pass**

   ```python
   xenc = F.one_hot(xs, num_classes=27).float()
   logits = xenc @ W
   counts = logits.exp()
   probs = counts / counts.sum(1, keepdims=True)
   ```

3. **Loss with regularisation**

   ```python
   loss = -probs[torch.arange(num), ys].log().mean() + 0.01 * (W**2).mean()
   ```

   - First term: average negative log‑likelihood (fit the data).
   - Second term: **weight decay** / L2 regularisation (keep weights small, avoid over‑confident weird spikes).

4. **Training loop**

   On each iteration:

   - Zero `W.grad`,
   - `loss.backward()`,
   - `W.data += -lr * W.grad`.

With a reasonably large learning rate (e.g. `50.0`) and ~50 steps, the loss should settle near the value you got from the pure counting model, showing that the neural network has **re‑discovered** the same bigram probabilities.


In [ ]:

# Exercise 11 – your turn: full dataset and training loop

### YOUR CODE HERE
raise NotImplementedError("Exercise 11: build full dataset and train W")

# Hints:
# - Follow the steps described in the markdown above.
# - Use a learning rate like 50.0 (as in the original notebook) so loss goes down quickly.


In [ ]:

# Solution 11 – full dataset and training loop

# 1. Build the dataset
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print("number of bigram examples:", num)

# 2. Initialise the network
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

# 3. Training loop
for k in range(50):
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float()  # one-hot inputs
    logits = xenc @ W                             # predict log-counts
    counts = logits.exp()                         # counts (unnormalised)
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character

    # loss: negative log likelihood + small weight decay
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01 * (W**2).mean()

    # backward pass
    W.grad = None
    loss.backward()

    # update
    lr = 50.0
    W.data += -lr * W.grad

    if k % 10 == 0 or k == 49:
        print(f"step {k:03d}  loss = {loss.item():.4f}")



## Exercise 12 – Sampling names from the trained neural model

After training, the learned weight matrix `W` defines a probability distribution over next characters,
very similar to our count‑based `P`, but now **learned** via gradient descent.

To sample names from the neural net model:

1. Start with `ix = 0` (for `'.'`).
2. For each step:
   - Build a one‑hot encoding for index `ix`.
   - Compute logits with `logits = xenc @ W`.
   - Turn logits into counts with `counts = logits.exp()`.
   - Normalise counts into probabilities `p = counts / counts.sum(1, keepdims=True)`.
   - Sample `ix` from `p` with `torch.multinomial`.
   - Append the character `itos[ix]` to the output list.
   - Stop when `ix == 0` again (end of word).

### Task

Using the trained `W` (from Exercise 11), sample and print 10 random names from the neural bigram model.

---
### Solution discussion

Sampling from the trained neural net mirrors what we did with the count‑based matrix:

For each new name:

1. Start with index `ix = 0` (the `'.'` start symbol).
2. Turn `ix` into a one‑hot vector and run it through the network:

   ```python
   xenc   = F.one_hot(torch.tensor([ix]), num_classes=27).float()
   logits = xenc @ W
   counts = logits.exp()
   p      = counts / counts.sum(1, keepdims=True)
   ```

3. Use `torch.multinomial(p, ...)` to sample the next index.
4. Append the character and repeat from step 2 with the new `ix`.
5. Stop when you sample `ix == 0` again (end of name).

Because `W` was trained to match the bigram statistics of the dataset, the names you get now should look **very similar** to the ones from the count‑based model — that’s a good sanity check that your training loop worked.


In [ ]:

# Exercise 12 – your turn: sampling from the neural net model

### YOUR CODE HERE
raise NotImplementedError("Exercise 12: sample names using trained W")

# Hints:
# - Use the loop described above.
# - Use a fixed generator seed for reproducibility.


In [ ]:

# Solution 12 – sampling from the neural net model

g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    ix = 0  # start from '.'
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))



## Wrap‑up

In this notebook you:

- Explored a dataset of baby names.
- Built a **bigram language model** by counting character pairs.
- Converted counts to probabilities and **sampled new names**.
- Measured model quality with **(average) negative log‑likelihood**.
- Reinterpreted the bigram model as a tiny **neural network**:
  - one‑hot inputs,
  - weight matrix `W`,
  - softmax outputs.
- Trained `W` with **gradient descent** using PyTorch autograd.
- Sampled new names from the **trained neural net**.

This is the conceptual foundation for much more powerful models:

- Replace "previous single character" with "previous N characters" (n‑grams).
- Replace one linear layer with deeper and more expressive networks.
- Replace characters with words, tokens, etc.

But the core ideas stay the same:

> Model defines probabilities for the next symbol →  
> We train parameters to make the training data **likely** under the model →  
> We can **sample** from the model to generate new, similar data.

From here, you can experiment with:

- Changing the vocabulary (uppercase, accent marks, etc.).
- Modifying the architecture (different hidden sizes, multiple layers).
- Comparing the count‑based bigram `P` and the learned `W` model quantitatively.

---
### Mental checklist of concepts you now own

Before moving on, make sure you’re comfortable with:

- Turning raw text into **bigrams** and counts.
- Converting counts → probabilities → samples.
- Computing **(average) negative log‑likelihood** as a loss.
- Viewing a table of probabilities as a **one‑layer neural network** with:
  - one‑hot inputs,
  - weight matrix `W`,
  - softmax outputs.
- Using PyTorch’s:
  - `F.one_hot`,
  - matrix multiplication (`@`),
  - `exp` and normalisation,
  - `loss.backward()` and manual weight updates.

In the next stages (multi‑layer MLPs, RNNs, and finally Transformers), only the **forward pass** gets more complicated. The training recipe — “compute logits → softmax → NLL loss → backward → update” — stays exactly the same.

If you really want to solidify this:

- Try changing the dataset (e.g. use a list of Pokémon names or movie titles).
- Swap the bigram model for a **trigram** model where you condition on the previous 2 characters.
- Compare the losses of the count‑based and learned models to make sure they match.

Once you’re happy with bigrams, you’re ready for much more powerful language models.
